In [ ]:
from KPrincipalGaussiansClassifier import KPrincipalGaussiansClassifier, MetaParams
import pandas as pd

def load_data(file_path): 
    print("loading data")
    all_data = pd.read_csv(file_path)
    train_data = all_data.to_numpy()[:, 1:]
    labels = all_data.to_numpy()[:, 0]
    print(f"train shape: {train_data.shape}")
    print(f"labels shape: {labels.shape}")
    
    return train_data, labels

train_data_file_path = 'C:/src/data_analysis/data/digit-recognizer/train.csv'
train_data, labels = load_data(train_data_file_path)
meta_params = MetaParams(num_classes=10,n_init=2,tol=1e-4,variance_perc_threshold=0.98,k_near_centroids=16) 

k_prin_gauss_model = KPrincipalGaussiansClassifier(meta_params=meta_params)
k_prin_gauss_model.fit(train_data=train_data, labels=labels)





loading data
train shape: (42000, 784)
labels shape: (42000,)
PCA k_seg_por_classe: 331
Treinando GaussianMixture para a classe: 0
Initialization 0
Initialization converged. time lapse 43.29717s	 lower bound -86502993.70166.
Treinando GaussianMixture para a classe: 1
Initialization 0
Initialization converged. time lapse 53.12515s	 lower bound -53487752.32521.
Treinando GaussianMixture para a classe: 2
Initialization 0
Initialization converged. time lapse 52.55458s	 lower bound -91793667.08225.
Treinando GaussianMixture para a classe: 3
Initialization 0
Initialization converged. time lapse 48.71820s	 lower bound -90887587.80867.
Treinando GaussianMixture para a classe: 4
Initialization 0
Initialization converged. time lapse 45.55505s	 lower bound -88035917.43640.
Treinando GaussianMixture para a classe: 5
Initialization 0
Initialization converged. time lapse 42.46476s	 lower bound -90459134.87653.
Treinando GaussianMixture para a classe: 6
Initialization 0
Initialization converged. time

In [ ]:
import numpy as np
np.save('mahalanobis', [k_prin_gauss_model], allow_pickle=True)

In [78]:
from search_optimization import create_hnsw_from_centroids, find_closest_segments
# loaded_arr = np.load('99_confidence.npy', allow_pickle=True)

import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from operation import euclidean_distance_to_segment

def validate_segments_with_weights(point, indices, variance_pow=0.5):
    """
    Valida os segmentos considerando as distâncias ajustadas com PCA2 e os pesos das gaussianas.
    
    segment_weights: array contendo os pesos das gaussianas associadas aos segmentos.
    """
    min_adjusted_distance = float('inf')
    best_index = -1

    for idx in indices:
        A, B = self.all_segments[idx]
        distance = euclidean_distance_to_segment(P=point, A=A, B=B)
        
        # Ajustar distância com PCA2 e peso gaussiano
        variance_weight = self.all_pc2_variances[idx] ** variance_pow
        adjusted_distance = distance / variance_weight
        
        if adjusted_distance < min_adjusted_distance:
            min_adjusted_distance = adjusted_distance
            best_index = idx

    return best_index
    
num_samples = 10000
validation_data = train_data[:num_samples,:]
labels_v = labels[:num_samples]
k_prin_gauss_model.k_near_centroids = 5
predictions = k_prin_gauss_model.predict(validation_data, variance_pow=0, weight_pow=0)
prev5_true3 = validation_data[(predictions == 5) & (labels_v == 3)]
print(prev5_true3)
p = k_prin_gauss_model.pca.transform(prev5_true3[0].reshape(1,-1))
distances_segs = []
pc2 = []
transformed = []
indices, distances = k_prin_gauss_model.hnsw.knn_query(p , k=5)  # Busca os k centroides mais próximos
classes = indices // k_prin_gauss_model.k_segments_per_class


info_point = np.vstack([classes, distances, distances_segs, pc2, transformed])
    
# # Compute confusion matrix
# cm = confusion_matrix(labels_v, predictions)

# # Display the confusion matrix
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot()

np.int64(331)

In [ ]:
''' Gerar submissão '''
loaded_arr = np.load('99_confidence.npy', allow_pickle=True)

kgm = loaded_arr[0]  
file_path = '../data/digit-recognizer/test.csv' 
test_data = pd.read_csv(file_path)
data_points = test_data.to_numpy()
 
test_data.loc[:, "Label"] = kgm.predict(data_points).astype(int)
answer = test_data[["Label"]].copy()  
answer["ImageId"] = range(1, len(answer) + 1)  
answer = answer[["ImageId", "Label"]]

# Save to CSV
output_file = "submission.csv"
answer.to_csv(output_file, index=False)

28000it [00:13, 2072.53it/s]
